<a href="https://colab.research.google.com/github/manisankar29/Lane_line_detection/blob/main/Lane_Line_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing necessary libraries**

In [1]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from moviepy import editor
import moviepy

## **Defining the driver function**

In [11]:
def process_video(test, output):
  input_video = editor.VideoFileClip(test, audio=False)
  processed = input_video.fl_image(frame_processor)
  processed.write_videofile(output, audio=False)

## **Defining "frame_processor" function where all the processing happens on a frame to detect lane lines**

In [17]:
def frame_processor(image):
  grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  kernel_size = 5
  blur = cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
  low_t = 50
  high_t = 150
  edges = cv2.Canny(blur, low_t, high_t)
  region = region_selection(edges)
  hough = hough_transform(region)
  result = draw_lane_lines(image, lane_lines(image, hough))
  return result

## **Defining "region_selection" function**

In [19]:
def region_selection(image):
  mask = np.zeros_like(image)

  if len(image.shape) > 2:
    channel_count = image.shape[2]
    ignore_mask_color = (255,) * channel_count
  else:
    ignore_mask_color = 255

  rows, cols = image.shape[:2]
  bottom_left  = [cols * 0.1, rows * 0.95]
  top_left     = [cols * 0.4, rows * 0.6]
  bottom_right = [cols * 0.9, rows * 0.95]
  top_right    = [cols * 0.6, rows * 0.6]
  vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)

  cv2.fillPoly(mask, vertices, ignore_mask_color)

  masked_image = cv2.bitwise_and(image, mask)

  return masked_image

## **Defining "hough_transform" funtion**

In [5]:
def hough_transform(image):
  rho = 1
  theta = np.pi/180
  threshold = 20
  minLineLength = 20
  maxLineGap = 500

  return cv2.HoughLinesP(image, rho=rho, theta=theta, threshold=threshold, minLineLength=minLineLength, maxLineGap=maxLineGap)

## **Defining "average_slope_intercept" function**

In [6]:
def average_slope_intercept(lines):
  left_lines    = []
  left_weights  = []
  right_lines   = []
  right_weights = []

  for line in lines:
    for x1, y1, x2, y2 in line:
      if x1 == x2:
        continue

      slope = (y2 - y1) / (x2 - x1)
      intercept = y1 - (slope * x1)
      length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))

      if slope < 0:
        left_lines.append((slope, intercept))
        left_weights.append((length))
      else:
        right_lines.append((slope, intercept))
        right_weights.append((length))

  left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
  right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None

  return left_lane, right_lane

## **Defining "pixel_points" function**

In [7]:
def pixel_points(y1, y2, line):
  if line is None:
    return None

  slope, intercept = line
  x1 = int((y1 - intercept) / slope)
  x2 = int((y2 - intercept) / slope)
  y1 = int(y1)
  y2 = int(y2)
  return ((x1, y1), (x2, y2))

## **Defining "lane_lines" function**

In [8]:
def lane_lines(image, lines):
  left_lane, right_lane = average_slope_intercept(lines)
  y1 = image.shape[0]
  y2 = y1 * 0.6
  left_line = pixel_points(y1, y2, left_lane)
  right_line = pixel_points(y1, y2, right_lane)
  return left_line, right_line

## **Defining "draw_lane_lines" function**

In [21]:
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
  line_image = np.zeros_like(image)
  for line in lines:
    if line is not None:
      cv2.line(line_image, *line, color, thickness)

  return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

## **Calling driver function**

In [22]:
process_video('challenge_video.mp4','output.mp4')

Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



t: 100%|██████████| 485/485 [00:53<00:00,  6.11it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file challenge_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame 484/485, at time 16.15/16.15 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output.mp4


In [23]:
process_video('challenge_video1.mp4','output1.mp4')

Moviepy - Building video output1.mp4.
Moviepy - Writing video output1.mp4



t: 100%|█████████▉| 1296/1297 [01:40<00:00,  8.61it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file challenge_video1.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1295/1298, at time 25.90/25.94 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output1.mp4
